## Description: Training and testing of a cnn based on the landmark data extracted from MediaPipe.
## Author: David Gray

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

In [ ]:
class_names = ["A","B","C","D","E","F","G","H","I","J","K","L","O","P","Q","R","T","V","W","Y"]
# class_names = ["A","B","C","D","E","F","H","I","K","L","O","P","Q","W"]
model_fp = "..\\models\\landmark_cnn_v2.h5"
# model_fp = "..\\models\\landmark_cnn_v4_best_letters.h5"
data_fp = "..\\parsed_dataset.csv"
train_fp = "..\\train.csv"
test_fp = "..\\test.csv"

In [ ]:
def createModel(num_classes):
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(5,5,3,)),
        layers.MaxPooling2D((2,2), padding='same'),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')])
    model.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

In [ ]:
def createDataset(fp):
    df = pd.read_csv(fp)
    lm_dataset = []
    label_dataset = []
    for i in range(len(df)):
        label = df.iloc[i,0]
        lm_list = []
        lm_group=[]
        for j in range(1,17):
            landmark = [df.iloc[i,j], df.iloc[i,j+21], df.iloc[i,j+42]]
            lm_group.append(landmark)
            if (j)%4==0:
                lm_list.append(lm_group)
                lm_group=[]
        for j in range(17,22):
            landmark = [df.iloc[i,j], df.iloc[i,j+21], df.iloc[i,j+42]]
            lm_group.append(landmark)
        lm_list.append(lm_group)
        lm_dataset.append(lm_list)
        label_dataset.append(label)
    features = tf.ragged.constant(lm_dataset)
    labels = tf.constant(label_dataset)
    dataset = tf.data.Dataset.from_tensor_slices((features.to_tensor(), labels))
    dataset = dataset.batch(batch_size=32)
    print("features:",features.shape)
    print("labels:",labels.shape)
    print("dataset:",dataset)
    return dataset

In [ ]:
def createSample(df):
    lm_dataset = []
    lm_list = []
    lm_group=[]
    for j in range(0,16):
        landmark = [df.iloc[j], df.iloc[j+21], df.iloc[j+42]]
        lm_group.append(landmark)
        if (j+1)%4==0:
            lm_list.append(lm_group)
            lm_group=[]
    for j in range(16,21):
        landmark = [df.iloc[j], df.iloc[j+21], df.iloc[j+42]]
        lm_group.append(landmark)
    lm_list.append(lm_group)
    lm_dataset.append(lm_list)
    features = tf.ragged.constant(lm_dataset)
    sample = features.to_tensor()
    return sample

In [ ]:
ds = pd.read_csv(data_fp)
train, test = train_test_split(ds, random_state=1, test_size=0.2)
print(len(train))
print(len(test))
train.to_csv(train_fp, index=False)
test.to_csv(test_fp, index=False)

In [ ]:
train_ds = createDataset(train_fp)
test_ds = createDataset(test_fp)

In [ ]:
model = createModel(len(class_names))
history = model.fit(train_ds, epochs=200, validation_data=test_ds)
model.summary()
history

In [ ]:
plt.figure(figsize=(10,10))
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
print(test_acc*100)

In [ ]:
model.save(model_fp)

In [ ]:
model = models.load_model(model_fp)

In [ ]:
print("----------TRAIN----------\n")
predict_train = []
y_train_values = []
train_df = pd.read_csv(train_fp)
for i in range(len(train_df)):
    sample_df = train_df.iloc[i]
    sample = createSample(sample_df[1:])
    pred = model.predict(sample)
    predict_train.append(np.argmax(pred[0]))
    y_train_values.append(sample_df[0])
print(confusion_matrix(y_train_values,predict_train))
print(classification_report(y_train_values,predict_train))
print("----------TEST----------\n")
predict_test = []
y_test_values = []
test_df = pd.read_csv(test_fp)
for i in range(len(test_df)):
    sample_df = train_df.iloc[i]
    sample = createSample(sample_df[1:])
    pred = model.predict(sample)
    predict_test.append(np.argmax(pred[0]))
    y_test_values.append(sample_df[0])
print(confusion_matrix(y_test_values,predict_test))
print(classification_report(y_test_values,predict_test))